#### Context: ABC Shipping is a International Shipment company with office across the globe. Numerous aspects of the business - customer and shipment - are captured. You are provided with 3 specific data as outlined below:
1. Mapped_LCL_Global_Flag_Full_1.zip: All transhipments originating from India
2. Master_WW_cleaned.zip: All transhipments that are likely to have a match with shipments originating from India
3. currency_global.csv: Currency Conversion table to convert local currency into USD or Euro

Business Problem: What is the end to end shipment level, identified by a combination of OriginalBL, HouseBL, profitability.

0. What are the steps you will undertake to understand the data
1. how will you identify and validate unique records in each file
2. Using a combination of OriginalBL and HouseBL in each file, match the records in each file. What data processing steps will you undertake to achieve the objective
3. How many unique shipments are there in the data. A shipment is a combination of OriginalBL and HouseBL
4. What is the total revenue, cost and profit
5. Combine Information from currency_global file into the merged entity
6. What are the outliers in the data wrt revenue, cost and profit
7. How many clusters can be created with these three variables.




In [1]:
import pandas as pd
import datetime
Mapped_LCL_Global_Flag_Full_1 = pd.read_csv(r'D:\Asignment\Mapped_LCL_Global_Flag_Full_1.csv')
Master_WW_cleaned = pd.read_csv(r'D:\Asignment\Master_WW_cleaned.csv')
currency_global = pd.read_excel(r'D:\Asignment\currency_global_map.xlsx')

C:\Users\Abhishek\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (20,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
Mapped_LCL_Global_Flag_Full_1.columns

Index(['CRM_ALLID', 'HouseBL', 'OriginalBL', 'Our_Customer', 'Our_CustomerID',
       'CompanyType', 'Confirmed_ETD', 'Actual_BL_POL', 'Actual_BL_POLIATA',
       'Actual_BL_POL_Country', 'Actual_BL_POL_Region', 'TradeLane_POL',
       'FiMN_Voyage_POL', 'FiMN_Voyage_POD', 'Tradelane_POD', 'Actual_BL_POD',
       'Actual_BL_PODIATA', 'Actual_BL_POD_Country', 'Actual_BL_POD_Region',
       'Final_Destination', 'Delivery', 'FreightPay', 'IMCO', 'CBM', 'KOS',
       'FT', 'FIMN_DATE', 'Bookdate', 'TOTAL_ACTREVS_for_BL',
       'TOTAL_ESTREVS_for_BL', 'TOTAL_ACTCOSTS_for_BL',
       'TOTAL_ESTCOSTS_for_BL', 'TOTAL_ACTPROFIT_for_BL',
       'TOTAL_ESTPROFIT_for_BL', 'YIELD', 'CURRENCY', 'IMEX'],
      dtype='object')

In [3]:
Master_WW_cleaned.columns 

Index(['Actual_BL_POD_Country', 'Actual_BL_POL_Country', 'CBM', 'CURRENCY',
       'Calc_Profit', 'Calc_Profit_USD', 'Calc_Yield', 'FILE_DATE', 'HouseBL',
       'IMEX', 'KOS', 'OriginalBL', 'Our_Customer', 'Source',
       'TOTAL_ACTREVS_for_BL', 'TOTAL_ESTCOSTS_for_BL', 'Rev_OriginalBL'],
      dtype='object')

In [4]:
currency_global.columns

Index(['ID', 'CountryOffice', 'Currency', 'Year', 'Period', 'RateUSD',
       'RateEUR', 'LegalEntityID', 'File_Source', 'List_office_country'],
      dtype='object')

### For understanding the data
    1. Gone through the glossary of file columns
         where POL : Point of landing
         where POD : Point of Deliever
        1. HouseBL:B/L issued by a freight forwarder or consolidator covering a single shipment containing the names,
           addresses and specific description of the goods shipped
        2. OriginalBL:The part of the B/L set that has value, especially when negotiable; rest of
           set are only informational file copies. Abbreviated as OBL
        3. Our_Customer: Customer Name
        4. Our_CustomerID: Customer ID
        5. Actual_BL_POLL Poart of Loading
        6. TradeLane_POL : A lane is another word for a “route” or the way in which cargo is transported around the world
        7. TOTAL_ACTREVS_for_BL : Actual revenue
        8. TOTAL_ACTCOSTS_for_BL : Actual Cost
        9. TOTAL_ACTPROFIT_for_BL : Actual Profit
        10.FreightPay: Type of Payment
        11.IMCO Codes are set by the International Maritime Organization for the safe transportation of dangerous goods by    vessel and to prevent marine pollution
        12.CBM stands for "cubic meter" in shipping. This measurement is calculated by multiplying the width, height and length together of one's carton.
        13.imex: Industrial Materials Exchange

In [5]:
len(Mapped_LCL_Global_Flag_Full_1)

209960

In [6]:
len(Master_WW_cleaned)

46937

In [9]:
len(Mapped_LCL_Global_Flag_Full_1)

187647

In [10]:
len(Master_WW_cleaned)

45196

### how will you identify and validate unique records in each file
A unique record will be based upon columns:
1. 'Our_CustomerID'
2. 'Actual_BL_POL'
3. 'TradeLane_POL'
4. 'Tradelane_POD'
5. 'Actual_BL_POD'
6. 'FreightPay'
7. 'TOTAL_ACTREVS_for_BL'
8. 'TOTAL_ACTCOSTS_for_BL'

In [7]:
#Finding the unique records in each file
Mapped_LCL_Global_Flag_Full_1 = Mapped_LCL_Global_Flag_Full_1.drop_duplicates(subset = ['Our_CustomerID','Actual_BL_POL','TradeLane_POL','Tradelane_POD','Actual_BL_POD','FreightPay','TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL']).copy()
Master_WW_cleaned = Master_WW_cleaned.drop_duplicates(subset = ['Actual_BL_POD_Country', 'Actual_BL_POL_Country','Calc_Profit','Calc_Yield' ,'Our_Customer','TOTAL_ACTREVS_for_BL','TOTAL_ESTCOSTS_for_BL']).copy()

In [8]:
len(Mapped_LCL_Global_Flag_Full_1)

188245

In [9]:
len(Master_WW_cleaned)

45210

### Using a combination of OriginalBL and HouseBL in each file, match the records in each file. What data processing steps will you undertake to achieve the objective

 1. I matched the OrignalBl with  it's HouseBL so as to match Records

In [10]:
Mapped_LCL_Global_Flag_Full_1[Mapped_LCL_Global_Flag_Full_1['HouseBL'].isin(list(Mapped_LCL_Global_Flag_Full_1['OriginalBL']))]

,CRM_ALLID,HouseBL,OriginalBL,Our_Customer,Our_CustomerID,CompanyType,Confirmed_ETD,Actual_BL_POL,Actual_BL_POLIATA,Actual_BL_POL_Country,...,Bookdate,TOTAL_ACTREVS_for_BL,TOTAL_ESTREVS_for_BL,TOTAL_ACTCOSTS_for_BL,TOTAL_ESTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL,TOTAL_ESTPROFIT_for_BL,YIELD,CURRENCY,IMEX
5,818938,HZR/KOP/00161,HZR/KOP/00161,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,NaN,HAZIRA,HZR,INDIA,...,6/25/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
8,823164,NAV/ALX/00923,NAV/ALX/00923,SUDARSHAN CHEMICAL INDUSTRIES LIMITED,1434401.0,Direct Account Direct Account,43377.0,NHAVA SHEVA,NAV,INDIA,...,9/27/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
9,823667,HZR/PUS/02981,HZR/PUS/02981,R N LABS -SEZ- NAV STARBOARD,39487.0,Direct Account Direct Account,43492.0,HAZIRA,HZR,INDIA,...,1/25/2019,43014.16,43014.16,41399.89,41399.89,1614.27,1614.27,0.00,INR,E
11,825018,NAV/ANR/03134,NAV/ANR/03134,HTL LOGISTICS NAV STARBOARD,34403.0,Forwarder General Forwarder,43296.0,NHAVA SHEVA,NAV,INDIA,...,7/9/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
12,826123,NAV/ATM/00076,NAV/ATM/00076,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43427.0,NHAVA SHEVA,NAV,INDIA,...,11/17/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
13,827143,NAV/AAR/01408,NAV/AAR/01408,ON TIME INTL NAV STARBOARD,36617.0,Forwarder General Forwarder,43569.0,NHAVA SHEVA,NAV,INDIA,...,4/10/2019,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
14,827550,NAV/AKL/00480,NAV/AKL/00480,MANILAL PATEL NAV,36852.0,Forwarder General Forwarder,43416.0,NHAVA SHEVA,NAV,INDIA,...,11/6/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
15,828246,NAV/BCN/01460,NAV/BCN/01460,BOLLORE LOGISTICS NAV,37835.0,Forwarder Global Account,43344.0,NHAVA SHEVA,NAV,INDIA,...,8/25/2018,NaN,NaN,NaN,NaN,NaN,NaN,0.00,INR,E
16,828664,HZR/KOP/00172,HZR/KOP/00172,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,43295.0,HAZIRA,HZR,INDIA,...,7/14/2018,172431.44,172431.44,167478.57,167478.57,4952.87,4952.87,0.00,INR,E
19,831159,NAV/APP/00990,NAV/APP/00990,KING FREIGHT MARITIME PVT LTD,1455622.0,Direct Account Direct Account,43481.0,NHAVA SHEVA,NAV,INDIA,...,1/14/2019,223587.00,223587.00,221813.00,221813.00,1774.00,1774.00,0.00,INR,E


In [11]:
Master_WW_cleaned[Master_WW_cleaned['HouseBL'].isin(list(Master_WW_cleaned['OriginalBL']))]

,Actual_BL_POD_Country,Actual_BL_POL_Country,CBM,CURRENCY,Calc_Profit,Calc_Profit_USD,Calc_Yield,FILE_DATE,HouseBL,IMEX,KOS,OriginalBL,Our_Customer,Source,TOTAL_ACTREVS_for_BL,TOTAL_ESTCOSTS_for_BL,Rev_OriginalBL
0,PAKISTAN,NETHERLANDS,2.080,EUR,146.01,165.983438,70.19711538,12/17/2018,RTMKHI02457,E,332.00,RTMKHI02457,ROTRA ROTTERDAM,MN13 - Netherland.csv,215.98,69.97,RTMKHI02457
1,PAKISTAN,NETHERLANDS,2.080,EUR,-124.99,-142.088007,-60.09134615,12/23/2018,RTMKHI02457,E,332.00,RTMKHI02457,ABC ROTTERDAM,MN24 - Belgium.csv,88.50,213.49,RTMKHI02457
2,BELGIUM,INDIA,13.954,EUR,-387.70,-440.735421,-27.78414791,12/16/2018,COKANR03655,I,5607.60,COKANR03655,ABC BELGIUM NV,MN13 - Netherland.csv,0.00,387.70,COKANR03655
3,BELGIUM,INDIA,13.954,EUR,506.39,575.661619,36.2899527,12/24/2018,COKANR03655,I,5607.60,COKANR03655,CLASQUIN LESQUIN,MN24 - Belgium.csv,1829.00,1322.61,COKANR03655
4,BELGIUM,INDIA,1.027,EUR,-30.38,-34.535832,-29.58130477,12/29/2018,MAABSL00161,I,1700.90,MAABSL00161,ABC BELGIUM NV,MN13 - Netherland.csv,0.00,30.38,MAABSL00161
5,SWITZERLAND,INDIA,1.027,EUR,-44.15,-50.189499,-42.98928919,12/29/2018,MAABSL00161,I,1700.90,MAABSL00161,EXPEDITORS TIC,MN24 - Belgium.csv,454.69,498.84,MAABSL00161
6,BELGIUM,INDIA,1.000,EUR,-34.43,-39.139852,-34.43,12/15/2018,MAABSL00158,I,222.00,MAABSL00158,ABC BELGIUM NV,MN13 - Netherland.csv,0.00,34.43,MAABSL00158
7,SWITZERLAND,INDIA,1.000,EUR,-17.01,-19.336883,-17.01,12/15/2018,MAABSL00158,I,222.00,MAABSL00158,ABCIMPORT,MN24 - Belgium.csv,223.66,240.67,MAABSL00158
8,BELGIUM,INDIA,3.342,EUR,-66.88,-76.028849,-20.01196888,12/22/2018,MAABSL00159,I,300.00,MAABSL00159,ABC BELGIUM NV,MN13 - Netherland.csv,0.00,66.88,MAABSL00159
9,SWITZERLAND,INDIA,3.342,EUR,-220.22,-250.344994,-65.89467385,12/22/2018,MAABSL00159,I,300.00,MAABSL00159,EXPEDITORS ZURICH,MN24 - Belgium.csv,391.34,611.56,MAABSL00159


### How many unique shipments are there in the data. A shipment is a combination of OriginalBL and HouseBL

In [12]:
print(len(Mapped_LCL_Global_Flag_Full_1[Mapped_LCL_Global_Flag_Full_1['HouseBL'].isin(list(Mapped_LCL_Global_Flag_Full_1['OriginalBL']))]))
print(len(Master_WW_cleaned[Master_WW_cleaned['HouseBL'].isin(list(Master_WW_cleaned['OriginalBL']))]))

169932
31182


### What is the total revenue, cost and profit

In [13]:
#cleaning the data
''' 
Dropping the nan values from columns 'TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL' for Mapped_LCL_Global_Flag_Full_1
'''
Mapped_LCL_Global_Flag_Full_1 =Mapped_LCL_Global_Flag_Full_1.dropna(subset=['TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL'])
Mapped_LCL_Global_Flag_Full_1.reset_index(drop=True,inplace=True)
Master_WW_cleaned =Master_WW_cleaned.dropna(subset=['Calc_Profit','TOTAL_ACTREVS_for_BL','Calc_Yield'])
Master_WW_cleaned.reset_index(drop=True,inplace=True)

In [14]:
print('Total_revunue')
print(sum(Mapped_LCL_Global_Flag_Full_1['TOTAL_ACTREVS_for_BL']))
print('Total Cost')
print(sum(Mapped_LCL_Global_Flag_Full_1['TOTAL_ACTCOSTS_for_BL']))
print('Total Profit')
print(sum(Mapped_LCL_Global_Flag_Full_1['TOTAL_ACTPROFIT_for_BL']))


Total_revunue
7546436241.990089
Total Cost
6234129296.330267
Total Profit
1312306945.6600113


### Combine Information from currency_global file into the merged entity

In [16]:
Master_WW_cleaned['FILE_DATE_Year']= Master_WW_cleaned['FILE_DATE'].apply(lambda x : int(pd.to_datetime(x,format='%m/%d/%Y').year))

In [17]:
Master_WW_cleaned['FILE_DATE_Month']= Master_WW_cleaned['FILE_DATE'].apply(lambda x : int(pd.to_datetime(x,format='%m/%d/%Y').month))

In [18]:
merge_entity = pd.merge(Master_WW_cleaned,currency_global,left_on = ['CURRENCY','Actual_BL_POD_Country','FILE_DATE_Year','FILE_DATE_Month'],right_on=['Currency','CountryOffice','Year','Period'],how='left')

In [19]:
merge_entity.head()

,Actual_BL_POD_Country,Actual_BL_POL_Country,CBM,CURRENCY,Calc_Profit,Calc_Profit_USD,Calc_Yield,FILE_DATE,HouseBL,IMEX,...,ID,CountryOffice,Currency,Year,Period,RateUSD,RateEUR,LegalEntityID,File_Source,List_office_country
0,PAKISTAN,NETHERLANDS,2.080,EUR,146.01,165.983438,70.19711538,12/17/2018,RTMKHI02457,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PAKISTAN,NETHERLANDS,2.080,EUR,-124.99,-142.088007,-60.09134615,12/23/2018,RTMKHI02457,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BELGIUM,INDIA,13.954,EUR,-387.70,-440.735421,-27.78414791,12/16/2018,COKANR03655,I,...,11536.0,BELGIUM,EUR,2018.0,12.0,0.879666,1.0,24.0,NaN,NaN
3,BELGIUM,INDIA,13.954,EUR,506.39,575.661619,36.2899527,12/24/2018,COKANR03655,I,...,11536.0,BELGIUM,EUR,2018.0,12.0,0.879666,1.0,24.0,NaN,NaN
4,BELGIUM,INDIA,1.027,EUR,-30.38,-34.535832,-29.58130477,12/29/2018,MAABSL00161,I,...,11536.0,BELGIUM,EUR,2018.0,12.0,0.879666,1.0,24.0,NaN,NaN


### What are the outliers in the data wrt revenue, cost and profit?
There are many way to find the Outliers in the Data
    1. Standard Deviation
    2. Boxplots
    3. DBScan Clustering
    4. outlier detection algorithms
        1. Angle-Based Outlier Detection (ABOD)
        2. k-Nearest Neighbors Detector
        3. Isolation Forest
        4. Local Correlation Integral (LOCI)
        5. Feature Bagging

My approach is using Isolation Forest to find the outliers in the data

Isolation Forest is an unsupervised learning algorithm that belongs to the ensemble decision trees family.
It explicitly isolates anomalies instead of profiling and constructing normal points and regions by assigning a score to each data point. It takes advantage of the fact that anomalies are the minority data points and that they have attribute-values that are very different from those of normal instances. This algorithm works great with very high dimensional datasets and it proved to be a very effective way of detecting anomalies

In [15]:
from sklearn.ensemble import IsolationForest
import numpy as np

### Fitting data
1. creating a 3 dimensions data which includes 'TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL'
2. creating a 1 dimensions data which includes 'TOTAL_ACTCOSTS_for_BL'
3. creating a 1 dimensions data which includes 'TOTAL_ACTREVS_for_BL'
4. creating a 1 dimensions data which includes 'TOTAL_ACTPROFIT_for_BL'

In [16]:
Mapped_LCL_Global_Flag_Full_1[['TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL']].values
random_data_cost = Mapped_LCL_Global_Flag_Full_1[['TOTAL_ACTCOSTS_for_BL']].values
random_data_rev = Mapped_LCL_Global_Flag_Full_1[['TOTAL_ACTREVS_for_BL']].values
random_data_profit = Mapped_LCL_Global_Flag_Full_1[['TOTAL_ACTPROFIT_for_BL']].values

In [32]:
clf = IsolationForest( behaviour = 'new', max_samples=100, random_state = 1, contamination= 'auto')

In [17]:
# If the result is -1, it means that this specific data point is an outlier. If the result is 1, then it means that the data point is not an outlier
preds = clf.fit_predict(random_data)
preds_cost = clf.fit_predict(random_data_cost)
preds_rev = clf.fit_predict(random_data_rev)
preds_profit = clf.fit_predict(random_data_profit)

In [31]:
Mapped_LCL_Global_Flag_Full_1['Outliers_wrt_3d'] = preds
Mapped_LCL_Global_Flag_Full_1['Outliers_wrt_cost'] = preds_cost
Mapped_LCL_Global_Flag_Full_1['Outliers_wrt_rev'] = preds_rev
Mapped_LCL_Global_Flag_Full_1['Outliers_wrt_profit'] = preds_rev

In [18]:
outliers =Mapped_LCL_Global_Flag_Full_1.copy()

In [33]:
outliers[outliers['Outliers_wrt_3d']== -1]

,Unnamed: 0,CRM_ALLID,HouseBL,OriginalBL,Our_Customer,Our_CustomerID,CompanyType,Confirmed_ETD,Actual_BL_POL,Actual_BL_POLIATA,...,TOTAL_ESTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL,TOTAL_ESTPROFIT_for_BL,YIELD,CURRENCY,IMEX,Outliers_wrt_3d,Outliers_wrt_cost,Outliers_wrt_rev,Outliers_wrt_profit
3,3,818562,HZR/CAS/00264,NAV/CAS/00263,GALLICTRANS CARGO NAV,36428.0,Forwarder General Forwarder,43151.0,HAZIRA,HZR,...,91408.00,3923.40,3923.40,0.00,INR,E,-1,-1,-1,-1
6,6,820873,CSI/TAS/00057,25050532565,ZANDRA LIFE SCIENCES PVT LTD,1462852.0,Direct Account Direct Account,NaN,MUMBAI AIRPORT,CSI,...,102172.70,103428.66,3802.66,0.00,INR,E,-1,1,-1,-1
9,9,828664,HZR/KOP/00172,HZR/KOP/00172,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,43295.0,HAZIRA,HZR,...,167478.57,4952.87,4952.87,0.00,INR,E,-1,-1,-1,-1
11,11,831159,NAV/APP/00990,NAV/APP/00990,KING FREIGHT MARITIME PVT LTD,1455622.0,Direct Account Direct Account,43481.0,NHAVA SHEVA,NAV,...,221813.00,1774.00,1774.00,0.00,INR,E,-1,-1,-1,-1
12,12,832089,NAV/BCN/01062,NAV/BCN/01062,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43250.0,NHAVA SHEVA,NAV,...,89682.53,6417.87,6417.87,0.00,INR,E,-1,-1,-1,-1
13,13,835235,NAV/APP/00759,NAV/APP/00759,EXPRESS SEAWAYS NAV,37972.0,Forwarder General Forwarder,43342.0,NHAVA SHEVA,NAV,...,176697.05,6877.35,6877.35,0.00,INR,E,-1,-1,-1,-1
14,14,837490,NAV/BCN/01952,NAV/BCN/01952,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43560.0,NHAVA SHEVA,NAV,...,58490.93,27988.63,27988.63,0.00,INR,E,-1,-1,-1,-1
18,18,848495,NAV/GOA/02732,NAV/GOA/02732,OCL SHIPPING DEL,40648.0,Forwarder General Forwarder,43490.0,NHAVA SHEVA,NAV,...,93153.54,17832.71,17832.71,0.00,INR,E,-1,-1,-1,-1
19,19,854641,NAV/HAM/04924,NAV/HAM/04924,VIGHNAHAR CMNARING NAV,33311.0,Forwarder General Forwarder,43116.0,NHAVA SHEVA,NAV,...,77334.80,2110.75,2110.75,0.00,INR,E,-1,-1,-1,-1
20,20,857289,NAV/HDD/00502,NAV/HDD/00502,VEDANTA LIMITED NAV,33028.0,Direct Account Direct Account,NaN,NHAVA SHEVA,NAV,...,6873.00,39784.00,39784.00,0.00,INR,E,-1,1,1,1


In [34]:
outliers[outliers['Outliers_wrt_cost']== -1]

,Unnamed: 0,CRM_ALLID,HouseBL,OriginalBL,Our_Customer,Our_CustomerID,CompanyType,Confirmed_ETD,Actual_BL_POL,Actual_BL_POLIATA,...,TOTAL_ESTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL,TOTAL_ESTPROFIT_for_BL,YIELD,CURRENCY,IMEX,Outliers_wrt_3d,Outliers_wrt_cost,Outliers_wrt_rev,Outliers_wrt_profit
2,2,818223,CSI/EBB/00046,17699874935,UNISYNTH CHEMICALS,1475988.0,Direct Account Direct Account,43456.0,MUMBAI AIRPORT,CSI,...,40490.75,1476.25,1476.25,0.00,INR,E,1,-1,1,1
3,3,818562,HZR/CAS/00264,NAV/CAS/00263,GALLICTRANS CARGO NAV,36428.0,Forwarder General Forwarder,43151.0,HAZIRA,HZR,...,91408.00,3923.40,3923.40,0.00,INR,E,-1,-1,-1,-1
4,4,818768,CSI/SUV/00031,58933048260,CONTECH LOGISTICS NAV,147259.0,Forwarder General Forwarder,43167.0,MUMBAI AIRPORT,CSI,...,47443.80,3055.20,3055.20,0.00,INR,E,1,-1,1,1
7,7,823667,HZR/PUS/02981,HZR/PUS/02981,R N LABS -SEZ- NAV STARBOARD,39487.0,Direct Account Direct Account,43492.0,HAZIRA,HZR,...,41399.89,1614.27,1614.27,0.00,INR,E,1,-1,1,1
9,9,828664,HZR/KOP/00172,HZR/KOP/00172,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,43295.0,HAZIRA,HZR,...,167478.57,4952.87,4952.87,0.00,INR,E,-1,-1,-1,-1
11,11,831159,NAV/APP/00990,NAV/APP/00990,KING FREIGHT MARITIME PVT LTD,1455622.0,Direct Account Direct Account,43481.0,NHAVA SHEVA,NAV,...,221813.00,1774.00,1774.00,0.00,INR,E,-1,-1,-1,-1
12,12,832089,NAV/BCN/01062,NAV/BCN/01062,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43250.0,NHAVA SHEVA,NAV,...,89682.53,6417.87,6417.87,0.00,INR,E,-1,-1,-1,-1
13,13,835235,NAV/APP/00759,NAV/APP/00759,EXPRESS SEAWAYS NAV,37972.0,Forwarder General Forwarder,43342.0,NHAVA SHEVA,NAV,...,176697.05,6877.35,6877.35,0.00,INR,E,-1,-1,-1,-1
14,14,837490,NAV/BCN/01952,NAV/BCN/01952,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43560.0,NHAVA SHEVA,NAV,...,58490.93,27988.63,27988.63,0.00,INR,E,-1,-1,-1,-1
16,16,844373,NAV/DMM/00784,NAV/DMM/00784,AL CAN EXPORTS NAV,1410396.0,Forwarder General Forwarder,43373.0,NHAVA SHEVA,NAV,...,68214.65,6568.75,6568.75,0.00,INR,E,1,-1,-1,-1


In [35]:
outliers[outliers['Outliers_wrt_rev']== -1]

,Unnamed: 0,CRM_ALLID,HouseBL,OriginalBL,Our_Customer,Our_CustomerID,CompanyType,Confirmed_ETD,Actual_BL_POL,Actual_BL_POLIATA,...,TOTAL_ESTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL,TOTAL_ESTPROFIT_for_BL,YIELD,CURRENCY,IMEX,Outliers_wrt_3d,Outliers_wrt_cost,Outliers_wrt_rev,Outliers_wrt_profit
3,3,818562,HZR/CAS/00264,NAV/CAS/00263,GALLICTRANS CARGO NAV,36428.0,Forwarder General Forwarder,43151.0,HAZIRA,HZR,...,91408.00,3923.40,3923.40,0.00,INR,E,-1,-1,-1,-1
6,6,820873,CSI/TAS/00057,25050532565,ZANDRA LIFE SCIENCES PVT LTD,1462852.0,Direct Account Direct Account,NaN,MUMBAI AIRPORT,CSI,...,102172.70,103428.66,3802.66,0.00,INR,E,-1,1,-1,-1
9,9,828664,HZR/KOP/00172,HZR/KOP/00172,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,43295.0,HAZIRA,HZR,...,167478.57,4952.87,4952.87,0.00,INR,E,-1,-1,-1,-1
11,11,831159,NAV/APP/00990,NAV/APP/00990,KING FREIGHT MARITIME PVT LTD,1455622.0,Direct Account Direct Account,43481.0,NHAVA SHEVA,NAV,...,221813.00,1774.00,1774.00,0.00,INR,E,-1,-1,-1,-1
12,12,832089,NAV/BCN/01062,NAV/BCN/01062,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43250.0,NHAVA SHEVA,NAV,...,89682.53,6417.87,6417.87,0.00,INR,E,-1,-1,-1,-1
13,13,835235,NAV/APP/00759,NAV/APP/00759,EXPRESS SEAWAYS NAV,37972.0,Forwarder General Forwarder,43342.0,NHAVA SHEVA,NAV,...,176697.05,6877.35,6877.35,0.00,INR,E,-1,-1,-1,-1
14,14,837490,NAV/BCN/01952,NAV/BCN/01952,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43560.0,NHAVA SHEVA,NAV,...,58490.93,27988.63,27988.63,0.00,INR,E,-1,-1,-1,-1
16,16,844373,NAV/DMM/00784,NAV/DMM/00784,AL CAN EXPORTS NAV,1410396.0,Forwarder General Forwarder,43373.0,NHAVA SHEVA,NAV,...,68214.65,6568.75,6568.75,0.00,INR,E,1,-1,-1,-1
17,17,846988,NAV/DUR/02757,NAV/DUR/02757,AIROSYS LOGISTICS NAV,1464687.0,Forwarder General Forwarder,43581.0,NHAVA SHEVA,NAV,...,74187.58,7691.28,4663.92,0.00,INR,E,1,-1,-1,-1
18,18,848495,NAV/GOA/02732,NAV/GOA/02732,OCL SHIPPING DEL,40648.0,Forwarder General Forwarder,43490.0,NHAVA SHEVA,NAV,...,93153.54,17832.71,17832.71,0.00,INR,E,-1,-1,-1,-1


In [36]:
outliers[outliers['Outliers_wrt_profit']== -1]

,Unnamed: 0,CRM_ALLID,HouseBL,OriginalBL,Our_Customer,Our_CustomerID,CompanyType,Confirmed_ETD,Actual_BL_POL,Actual_BL_POLIATA,...,TOTAL_ESTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL,TOTAL_ESTPROFIT_for_BL,YIELD,CURRENCY,IMEX,Outliers_wrt_3d,Outliers_wrt_cost,Outliers_wrt_rev,Outliers_wrt_profit
3,3,818562,HZR/CAS/00264,NAV/CAS/00263,GALLICTRANS CARGO NAV,36428.0,Forwarder General Forwarder,43151.0,HAZIRA,HZR,...,91408.00,3923.40,3923.40,0.00,INR,E,-1,-1,-1,-1
6,6,820873,CSI/TAS/00057,25050532565,ZANDRA LIFE SCIENCES PVT LTD,1462852.0,Direct Account Direct Account,NaN,MUMBAI AIRPORT,CSI,...,102172.70,103428.66,3802.66,0.00,INR,E,-1,1,-1,-1
9,9,828664,HZR/KOP/00172,HZR/KOP/00172,GUJARAT BOROSIL NAV,1431120.0,Forwarder General Forwarder,43295.0,HAZIRA,HZR,...,167478.57,4952.87,4952.87,0.00,INR,E,-1,-1,-1,-1
11,11,831159,NAV/APP/00990,NAV/APP/00990,KING FREIGHT MARITIME PVT LTD,1455622.0,Direct Account Direct Account,43481.0,NHAVA SHEVA,NAV,...,221813.00,1774.00,1774.00,0.00,INR,E,-1,-1,-1,-1
12,12,832089,NAV/BCN/01062,NAV/BCN/01062,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43250.0,NHAVA SHEVA,NAV,...,89682.53,6417.87,6417.87,0.00,INR,E,-1,-1,-1,-1
13,13,835235,NAV/APP/00759,NAV/APP/00759,EXPRESS SEAWAYS NAV,37972.0,Forwarder General Forwarder,43342.0,NHAVA SHEVA,NAV,...,176697.05,6877.35,6877.35,0.00,INR,E,-1,-1,-1,-1
14,14,837490,NAV/BCN/01952,NAV/BCN/01952,PRATIK SHIPPING NAV STARBOARD,38774.0,Forwarder General Forwarder,43560.0,NHAVA SHEVA,NAV,...,58490.93,27988.63,27988.63,0.00,INR,E,-1,-1,-1,-1
16,16,844373,NAV/DMM/00784,NAV/DMM/00784,AL CAN EXPORTS NAV,1410396.0,Forwarder General Forwarder,43373.0,NHAVA SHEVA,NAV,...,68214.65,6568.75,6568.75,0.00,INR,E,1,-1,-1,-1
17,17,846988,NAV/DUR/02757,NAV/DUR/02757,AIROSYS LOGISTICS NAV,1464687.0,Forwarder General Forwarder,43581.0,NHAVA SHEVA,NAV,...,74187.58,7691.28,4663.92,0.00,INR,E,1,-1,-1,-1
18,18,848495,NAV/GOA/02732,NAV/GOA/02732,OCL SHIPPING DEL,40648.0,Forwarder General Forwarder,43490.0,NHAVA SHEVA,NAV,...,93153.54,17832.71,17832.71,0.00,INR,E,-1,-1,-1,-1


### How many clusters can be created with these three variables.?
Clustering results with total revenue, customer count, total profit
    1. High Value:(High revunue, Low Cost) Strategy : Nurture
    2. Medium value : (High Revunue, High Cost) Strategy : Grow and Reduce Cost
    3. Low Value : (Low revunue,Low cost) Strategy : Grow Revunue
    4. Negative Value : (Low Revunue,High Cost) Strategy : Migrate Reject

In [40]:
Mapped_LCL_Global_Flag_Full_1[['Our_Customer','TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL']].groupby(["Our_Customer"]).sum().sort_values(["TOTAL_ACTREVS_for_BL","TOTAL_ACTCOSTS_for_BL"], ascending=[False,True])

,TOTAL_ACTREVS_for_BL,TOTAL_ACTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL
Our_Customer,,,
XYZ MAA,1.801096e+08,3.438650e+07,1.457231e+08
XYZ DEL,1.765750e+08,1.653875e+08,1.118752e+07
XYZ NAV,1.656552e+08,7.804938e+07,8.760586e+07
AUROBINDO PHARMA (UNIT-3),1.292267e+08,1.244746e+08,4.752048e+06
XYZ BLR,1.140955e+08,1.085547e+08,5.540792e+06
GARWARE WALL PNQ,1.089131e+08,1.017601e+08,7.152968e+06
ALKEM LABORATORIESLIMITED NAV,8.411100e+07,8.169644e+07,2.414559e+06
HINDUSTAN GUMS JPU,7.365299e+07,7.065894e+07,2.994047e+06
PRATIK SHIPPING NAV STARBOARD,4.911837e+07,4.090263e+07,8.215736e+06


In [42]:
Mapped_LCL_Global_Flag_Full_1[['Our_Customer','TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL']].groupby(["Our_Customer"]).sum().sort_values(["TOTAL_ACTREVS_for_BL","TOTAL_ACTCOSTS_for_BL"], ascending=[True,True])

,TOTAL_ACTREVS_for_BL,TOTAL_ACTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL
Our_Customer,,,
RUSHIL DECOR LTD GUJ,-1.819760e+03,-3.005180e+03,1.185420e+03
CENTRE FOR DEVELOPMENT OF TEMNMATICS,0.000000e+00,-9.517873e+04,9.517873e+04
SRI VINAYA TOOLS,0.000000e+00,-5.305348e+04,5.305348e+04
LPS BOSSARD PVT LTD,0.000000e+00,-1.486588e+04,1.486588e+04
LCL LOGISTIX INDIA LUH,0.000000e+00,-1.062500e+04,1.062500e+04
BOUNCING GREEN,0.000000e+00,-8.859310e+03,8.859310e+03
APTAR BEAUTY NAV,0.000000e+00,-7.710570e+03,7.710570e+03
SME VILMN PARMN FOREX MAH NAV,0.000000e+00,-6.765690e+03,6.765690e+03
RAUNAQ INTL NAV,0.000000e+00,-6.570050e+03,6.570050e+03


In [43]:
Mapped_LCL_Global_Flag_Full_1[['Our_Customer','TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL']].groupby(["Our_Customer"]).sum().sort_values(["TOTAL_ACTREVS_for_BL","TOTAL_ACTCOSTS_for_BL"], ascending=[True,False])

,TOTAL_ACTREVS_for_BL,TOTAL_ACTCOSTS_for_BL,TOTAL_ACTPROFIT_for_BL
Our_Customer,,,
RUSHIL DECOR LTD GUJ,-1.819760e+03,-3.005180e+03,1.185420e+03
GLOBAL AGENCIES,0.000000e+00,8.798697e+04,-8.798697e+04
SAMNAV TECHNOLOGIES,0.000000e+00,6.810367e+04,-6.810367e+04
SANDEN VIKAS INDIA PVT LTD,0.000000e+00,3.103045e+04,-3.103045e+04
A & G TECHNOLOGIES LUH,0.000000e+00,2.671801e+04,-2.671801e+04
DENSO TEN MINDA HAR NAV,0.000000e+00,2.171888e+04,-2.171888e+04
ACME FASTENERS PVT LTD,0.000000e+00,1.719237e+04,-1.719237e+04
INTABCAL INTERNATIONAL DEL NAV,0.000000e+00,1.463017e+04,-1.463017e+04
KN BIOSCIENCE INDIA PVT LTD TEL NAV,0.000000e+00,1.462510e+04,-1.462510e+04


### Top Customers with the most shipment

In [22]:
Mapped_LCL_Global_Flag_Full_1["Our_Customer"].value_counts().nlargest(10)

XYZ DEL           12576
XYZ BLR            8556
EXPEDITORS NAV     3450
EXPEDITORS MAA     2025
XYZ MAA            1990
XYZ AMD            1989
XYZ HYD            1799
XYZ LUH            1743
EXPEDITORS BLR     1357
EXPEDITORS DEL     1195
Name: Our_Customer, dtype: int64

### What Customer routes have the maximum profit,cost and revunue ?

In [44]:
group = Mapped_LCL_Global_Flag_Full_1[['Actual_BL_POL_Country','Actual_BL_POD_Country','Our_Customer','TOTAL_ACTREVS_for_BL','TOTAL_ACTCOSTS_for_BL','TOTAL_ACTPROFIT_for_BL','TradeLane_POL','Tradelane_POD']].groupby(['Our_Customer','Actual_BL_POL_Country','TradeLane_POL','Tradelane_POD','Actual_BL_POD_Country']).sum()
total_profit = group["TOTAL_ACTPROFIT_for_BL"].groupby(level=0, group_keys=False)
total_profit.nlargest(5)

Our_Customer                               Actual_BL_POL_Country  TradeLane_POL  Tradelane_POD  Actual_BL_POD_Country
  (ARAMEX) GLOBAL TRANSPORTATION SERVICE   INDIA                  NAV            PUS            CANADA                     9350.85
                                                                                 CMB            SRI LANKA                  3382.43
 ABITO KREATION NAV                        BELGIUM                CMB            CCU            INDIA                     24665.40
 ADVANCE SYNTEX LIMITED                    INDIA                  NAV            STP            RUSSIAN FEDERATION        10582.02
 AFRO ASIATIC EXPORTERS                    INDIA                  DEL            MDE            COLOMBIA                  18338.65
                                                                                 ZAG            CROATIA                   12028.40
                                                                                 MIA            

In [45]:
total_profit = group["TOTAL_ACTREVS_for_BL"].groupby(level=0, group_keys=False)
total_profit.nlargest(5)

Our_Customer                               Actual_BL_POL_Country  TradeLane_POL  Tradelane_POD  Actual_BL_POD_Country
  (ARAMEX) GLOBAL TRANSPORTATION SERVICE   INDIA                  NAV            PUS            CANADA                    39461.49
                                                                                 CMB            SRI LANKA                  5234.85
 ABITO KREATION NAV                        BELGIUM                CMB            CCU            INDIA                     49676.33
 ADVANCE SYNTEX LIMITED                    INDIA                  NAV            STP            RUSSIAN FEDERATION        27154.30
 AFRO ASIATIC EXPORTERS                    INDIA                  DEL            MDE            COLOMBIA                 515296.39
                                                                                 MIA            U.S.A.                   306480.00
                                                                                 ZAG            

In [46]:
total_profit = group["TOTAL_ACTCOSTS_for_BL"].groupby(level=0, group_keys=False)
total_profit.nlargest(5)

Our_Customer                               Actual_BL_POL_Country  TradeLane_POL  Tradelane_POD  Actual_BL_POD_Country
  (ARAMEX) GLOBAL TRANSPORTATION SERVICE   INDIA                  NAV            PUS            CANADA                    30110.64
                                                                                 CMB            SRI LANKA                  1852.42
 ABITO KREATION NAV                        BELGIUM                CMB            CCU            INDIA                     25010.93
 ADVANCE SYNTEX LIMITED                    INDIA                  NAV            STP            RUSSIAN FEDERATION        16572.28
 AFRO ASIATIC EXPORTERS                    INDIA                  DEL            MDE            COLOMBIA                 496957.74
                                                                                 MIA            U.S.A.                   295298.00
                                                                                 ZAG            